##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Salvataggio e caricamento di modelli

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Visualizza su TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/it/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Esegui in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/it/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Visualizza il sorgente su GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/it/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Scarica il notebook</a>
  </td>
</table>

Note: Questi documenti sono stati tradotti dalla nostra comunità di TensorFlow. Poichè queste traduzioni sono *best-effort*, non è garantito che rispecchino in maniera precisa e aggiornata la [documentazione ufficiale in inglese](https://www.tensorflow.org/?hl=en). 
Se avete suggerimenti per migliorare questa traduzione, mandate per favore una pull request al repository Github [tensorflow/docs](https://github.com/tensorflow/docs). 
Per proporsi come volontari alla scrittura o alla review delle traduzioni della comunità contattate la 
[mailing list docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

I cambiamenti dei modelli durante-edopo-l'addestramento possono essere salvati. Ciò significa che un modello può riprendere da dove aveva lasciato, ed evitare lunghi tempi di addestramento. Il salvataggio significa anche che potete condividere il vostro modello ed altri possono ricreare il vostro lavoro. La maggioranza di chi lavora nel machine learning, quando pubblica modelli e tecniche di ricerca, condivide:

* il codice per creare il modello, and
* i trained weights, o i parametri, per il modello

Condividere questi dati aiuta gli altri a capire come lavora il modello e provarlo in autonomia con nuovi dati.

Attenzione: Siate cauti con il codice non fidato—i modelli TensorFlow sono codice. Per i dettagli vedere [Uso sicuro di TensorFlow](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md).

### Opzioni

Ci sono diversi modi per salvare i modelli TensorFlow—a seconda delle API che state usando. Questa guida usa [tf.keras](https://www.tensorflow.org/guide/keras), un'API di alto livello per costruire e addestrare modelli in TensorFlow. Per altri approcci, vedere la guida [Salvataggio e Ripristino](https://www.tensorflow.org/guide/saved_model) di TensorFlow o [Salvataggio come eager](https://www.tensorflow.org/guide/eager#object-based_saving).

## Setup

### Installazioni e importazioni

Installare e importare TensorFlow e dipendenze:

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

### Procuriamoci un dataset di esempio

Per dimostrare come salvare e caricare weights, usiamo il [dataset MNIST](http://yann.lecun.com/exdb/mnist/). Per velocizzare il processo, usiamo i primi 1000 esempi:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Definiamo un modello

Cominciamo costruendo un semplice modello sequenziale:

In [ ]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

## Salvare i punti di controllo durante l'addestramento

Potete usare un modello addestrato senza doverlo addestrare di nuovo, o riprendere l'addestramento da dove avevate lasciato, nel caso il processo di addestramento sia stato interrotto. La callback `tf.keras.callbacks.ModelCheckpoint` permette di salvare il modello continuamente sia *durante* sia *alla fine* dell'addestramento.

### Utilizzo della callback Checkpoint

Creiamo una callback `tf.keras.callbacks.ModelCheckpoint` che salvi i weights solo durante l'addestramento:

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Ciò crea un'unica collezione di file con i punti di controllo di TensorFlow che vengono salvati alla fine di ogni epoca:

In [ ]:
!ls {checkpoint_dir}

Creiamo un nuovo modello, non addestrato. Quando ripristinate un modello dai soli weight, dovete avere un modello con la stessa architettura dell'originale. Dato che l'architettura del modello è la stessa, potete condividere gli weight anche se si tratta di una diversa *istanza* del modello.

Ora ricostruiamo una nuova istanza, non addestrata, del modello e valutiamola sull'insieme di test. Un modello non addestrato avrà prestazioni di livello basso (~10% di accuratezza):

In [ ]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

Ora carichiamo i pesi dal punto di controllo e valutiamo di nuovo:

In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

### Opzioni di callback del punto di controllo

La callback mette a disposizione diverse opzioni per dare un nome univoco ai punti di controllo e regolare la loro frequenza.

Addestriamo un nuovo modello, e salviamo i checkpoint con un nome univoco ogni cinque epoche:

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)

Ora, guardiamo i punti di controllo che abbiamo ottenuto e prendiamo l'ultimo:

In [ ]:
!ls {checkpoint_dir}

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

Notare: che il formato standard di tensorflow salva solo i 5 punti di controllo più recenti.

Per verificare, riavviamo il modello e carichiamo il checkpoint più recente:

In [ ]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Cosa sono questi file?

Il codice di cui sopra memorizza i pesi in una collezione di file formattati come [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-che contengono solo i pesi addestrati in un formato binario. I checkpoint contengono:
* Uno o più blocchi che contengono i pesi del nostro modello.
* Un file indice che indica quali pesi sono memorizzati in un determinato blocco.

Se state solo addestrando un modello su una singola macchina, avrete un blocco con il suffisso: `.data-00000-of-00001`

## Salvare i pesi manualmente

Avete visto come caricare i pesi in un modello. E' semplice salvarli manualmente con il metodo `Model.save_weights`. Per default, in particolare-`tf.keras`—and `save_weights` —usa il formato TensorFlow [checkpoint](../../guide/checkpoint.ipynb) con estensione `.ckpt` (il salvataggio in [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) con estensione `.h5` è trattato nella guida[Salvare e serializzare i modelli](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format)):

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Salvare l'intero modello

Per salvare l'architettura di un modello, i pesi, e la configurazione di addestramento in un singolo file/cartella, occorre chiamare [`model.save`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save). Ciò vi permette di esportare un modello in modo che possa essere usato senza accedere al codice* Python originale. Dato che viene ripristinato lo stato dell'ottimizzatore, potrete riprendere l'addestramento esattamente da dove l'avevate lasciato.

Un intero modello può essere salvato in due formati di file diversi (SavedModel e HDF). Occorre notare che il formato TensodrFlow SavedModel è il default in TF2.x. Comunque, un modello può essere salvato anche in formato HDF5. Maggiori dettagli sul salvataggio dei modelli nei due formati sono dati in seguito.

Il salvataggio di un modello completamente funzionante è molto utile—lo potete caricare in un TensorFlow.js ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Modello Salvato](https://js.tensorflow.org/tutorials/import-saved-model.html)) e poi addestrarlo ed eseguirlo in un browser web, o convertirlo per eseguirlo su un dispositivo mobile usando TensorFlow Lite ([HDF5](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Modello Salvato](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))

\*Oggetti personalizzati (es. modelli specializzati o livelli) richiedono particolare attenzione durante il salvataggio e il caricamento. Vedere la sezione **Salvataggio di oggetti personalizzati** più sotto

### formato HDF5

Keras fornisce un formato base di salvataggio che usa lo standard [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). 

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Ora, ri-creiamo il modello dal file:

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

E controlliamo la sua accuratezza:

In [ ]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

Questa tecnica salva ogni cosa:

* I valori dei pesi
* L'architettura del modello
* La configurazione di addestramento del modello (ciò che avete passato al compilatore)
* L'ottimizzatore ed il suo stato, se ce n'è uno (questo è ciò che vi permette di riprendere l'addestramento da dove l'avete lasciato)

Keras salva i modelli ispezionando l'architettura. Al momento, esso non è in grado di salvare gli ottimizzatori 'v1.x' (da `tf.compat.v1.train`) in quanto essi non sono compatibili con i punti di controllo. Quando li doveste usare, avrete bisogno di ri-compilare il modello dopo il caricamento, e perderete lo stato dell'ottimizzatore.


### formato SavedModel

Il formato SavedModel è un'altro modo di serializzare i modelli. I modelli salvati in questo formato possono essere ripristinati usando `tf.keras.models.load_model` e sono compatibili con TensorFlow Serving. La [guida SavedModel](https://www.tensorflow.org/guide/saved_model) scende nei dettagli su come utilizzare/ispezionare il SavedModel. La sezione che segue illustra i passi di salvataggio e recupero del modello.

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Il formato SavedModel è una directory che contiene un binario protobuf ed un checkpoint Tensorflow. Osserviamo la directory del modello salvato:

In [ ]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

Carichiamo un nuovo modello Keras dal modello salvato:

In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Il modello ripristinato viene compilato con gli stessi argomenti del modello originale. Proviamo ad eseguire, valutare e predire con il modello caricato:

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

### Salvataggio di oggetti personalizzati

Se usate il formato SavedModel potete saltare questa sezione. La differenza chiave tra HDF5 e SavedModel è che HDF5 usa le gli oggetti configs per salvare l'architettura del modello, mentre SavedModel salva il grafo di esecuzione. Così, i SavedModel sono in grado di salvare oggetti personalizzati come modelli derivati per specializzazione e livelli personalizzati senza bisogno del codice originale.

Per salvare oggetti personalizzati in HDF5, dovete:

1. Definire un metodo `get_config` nel vostro oggetto, e and facoltativamente un classmethod `from_config`.
  * `get_config(self)` ritorna un dizionario JSON-serializzabile dei parametri necessari per ri-creare l'oggetto.
  * `from_config(cls, config)` usa il config restituito da `get_config` per creare un nuovo oggetto. Per, questa funzione userà il config come kwargs di inizializzazioone (`return cls(**config)`).
2. Passare l'oggetto come argomento `custom_objects` al caricamento del modello. L'argomento deve essere un dizionario che mappi la stringa del nome della classe name nella classe Python. Es. `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

Vedere il tutorial [Scrivere livelli e modelli da zero](https://www.tensorflow.org/guide/keras/custom_layers_and_models) per esempi di oggetti personalizzati e `get_config`.
